# Imports

In [1]:
#from pandasai.llm.openai import OpenAI
#from dotenv import load_dotenv
import os
import pandas as pd
from pandasai import PandasAI

In [2]:
from langchain import OpenAI, SerpAPIChain,LLMChain


In [3]:
from pandasai import SmartDataframe

In [4]:
from pandasai.callbacks import StdoutCallback

In [5]:
from langchain import PromptTemplate


In [6]:
from langchain import OpenAI, SerpAPIChain,LLMChain


In [8]:
import openai

In [16]:
from langchain.agents import Tool,AgentExecutor,LLMSingleActionAgent,AgentOutputParser,create_csv_agent


# creds


In [9]:
import sys
sys.path.append('/home/ubuntu/workspace/Creds')
from openai_config import OPENAI_API_KEY, MW_ENVIRONMENT, MW_PINECONE_API_KEY
openai.api_key = OPENAI_API_KEY

# Paths

In [59]:
pth = '/home/ubuntu/workspace/Temp/07-14Oct/VFX_Shots_08-28-23.csv'

In [60]:
df  = pd.read_csv(pth, header = 0)

In [61]:
df.columns = df.iloc[0]
df = df[1:]


In [40]:
df = df.drop(columns=[col for col in df if col is None or pd.isna(col)])
df.columns

Index([              'Id',        'Thumbnail',        'Shot Code',
                 'Status',      'Description',             'Type',
                   'Misc',   'Creative Brief',             'Reel',
          'Vendor Groups',         'Priority', 'Open Notes Count',
       'Version <-> Link',               'Id',           'Status',
              'Task Name',      'Assigned To',       'Start Date',
               'Due Date',         'Duration',         'Turnover',
               'Sequence',    'Task Template',          'Project',
                      nan],
      dtype='object', name=0)

In [62]:
base_df = df.iloc[:, :-1]
base_df

,Id,Thumbnail,Shot Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor Groups,...,Status,Task Name,Assigned To,Start Date,Due Date,Duration,Turnover,Sequence,Task Template,Project
1,1189,NaN,LA_0070,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
2,1190,NaN,LA_0080,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
3,1191,NaN,LA_0100,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
4,1193,NaN,LA_0120,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
5,1295,https://misterwhisper.shotgrid.autodesk.com/th...,CR_0350,wtg,2D Shadow comp of MW in BG,2D Comp,NaN,MW shadow continues to grow over the last pane...,2,"Eye-Spy VFX, In-house",...,wtg,Roto,NaN,NaN,NaN,NaN,No,CR - Chaz Rescue,2D Comp,Mister Whisper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",...,wtg,Roto,NaN,NaN,NaN,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3636,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",...,wtg,Comp,NaN,NaN,NaN,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3637,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",...,wtg,Depth Map,NaN,NaN,NaN,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3638,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",...,wtg,Animation,NaN,NaN,NaN,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper


In [63]:
base_df.columns

Index(['Id', 'Thumbnail', 'Shot Code', 'Status', 'Description', 'Type', 'Misc',
       'Creative Brief', 'Reel', 'Vendor Groups', 'Priority',
       'Open Notes Count', 'Version <-> Link', 'Id', 'Status', 'Task Name',
       'Assigned To', 'Start Date', 'Due Date', 'Duration', 'Turnover',
       'Sequence', 'Task Template', 'Project'],
      dtype='object', name=0)

In [64]:
base_df.rename(columns={'Assigned To':"Assignee", 'Shot Code': 'Shot-Code',
                       'Version <-> Link': 'Version-Link', 'Due Date':'Due-Date',
                       'Task Template': 'Task-Template', 'Vendor Groups':'Vendor-Groups'}, inplace = True)

In [65]:
base_df.to_csv('/home/ubuntu/workspace/Temp/07-14Oct/processed_test_df.csv', index = False)

In [33]:
base_df = pd.read_csv('/home/ubuntu/workspace/Temp/07-14Oct/processed_test_df.csv')

In [34]:
base_df['Task-Template'].value_counts()

2D Comp         982
3D Comp - L2    625
3D Comp - L4    540
3D Comp - L3    501
3D Comp - L1    418
Name: Task-Template, dtype: int64

In [37]:
base_df['Creative Brief'].value_counts()

Eels wrap around the cops arm and make him shoot the gun                                               24
WHAT ARE WE DOING HERE?                                                                                22
Good for MW shadow mockup                                                                              20
NEEDS TEMP                                                                                             20
Eels jump onto cop one sync with Jacob's hand                                                          16
                                                                                                       ..
Footage needs to look messed up and loz rez on purpose                                                  1
Remove wire and fix the timming on the Red nose light to match the scare moment.                        1
Darken right side to match previous cut                                                                 1
Look a crew member in the back of the glass an

# Work

In [49]:
llm = OpenAI(api_token=OPENAI_API_KEY)
pandas_ai = PandasAI(llm)

/home/ubuntu/.pyenv/versions/langchain-3.10.2/lib/python3.10/site-packages/pandasai/__init__.py:145: UserWarning: `PandasAI` (class) is deprecated since v1.0 and will be removed in a future release. Please use `SmartDataframe` instead.
  warnings.warn(


In [50]:
df = SmartDataframe(base_df, {"callback": StdoutCallback()}, config={"llm": llm})

In [45]:
df.chat('Describe the columns, by using df.columns',)

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'DataFrame' object has no attribute 'dtype'\n"

In [51]:
df.chat('What is the value count in column Assigned To')

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'DataFrame' object has no attribute 'dtype'\n"

# RoughWork

In [15]:
df.head(2)

,Id,Thumbnail,Shot Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor Groups,...,Task Name,Assigned To,Start Date,Due Date,Duration,Turnover,Sequence,Task Template,Project,NaN
1,1189,NaN,LA_0070,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper,NaN
2,1190,NaN,LA_0080,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",...,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper,NaN


In [55]:
prompt = 'check the dtypes of df, by df.dtypes'

In [56]:
result = pandas_ai.run(base_df, prompt=prompt)

In [57]:
result

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'DataFrame' object has no attribute 'dtype'\n"

In [23]:
df.columns

Index([              'Id',        'Thumbnail',        'Shot Code',
                 'Status',      'Description',             'Type',
                   'Misc',   'Creative Brief',             'Reel',
          'Vendor Groups',         'Priority', 'Open Notes Count',
       'Version <-> Link',               'Id',           'Status',
              'Task Name',      'Assigned To',       'Start Date',
               'Due Date',         'Duration',         'Turnover',
               'Sequence',    'Task Template',          'Project',
                      nan],
      dtype='object', name=0)

In [24]:
df.drop(columns = ['nan'], inplace = True)

KeyError: "['nan'] not found in axis"

In [58]:
counts = df['Assigned To'].value_counts()
print(counts)


Ratheesh Vasu      160
Mike Linman         75
Jose Marra          43
In-house            23
Eye-Spy VFX         13
Robert Glickert      4
Jose VFX             1
Merrin Marra         1
George Moise         1
Name: Assigned To, dtype: int64


## With langchain

In [39]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3639 entries, 0 to 3638
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3639 non-null   int64  
 1   Thumbnail         3588 non-null   object 
 2   Shot-Code         3639 non-null   object 
 3   Status            3623 non-null   object 
 4   Description       3516 non-null   object 
 5   Type              3639 non-null   object 
 6   Misc              171 non-null    object 
 7   Creative Brief    691 non-null    object 
 8   Reel              3639 non-null   int64  
 9   Vendor-Groups     3639 non-null   object 
 10  Priority          1924 non-null   float64
 11  Open Notes Count  3639 non-null   int64  
 12  Version-Link      3588 non-null   object 
 13  Id.1              3103 non-null   float64
 14  Status.1          3103 non-null   object 
 15  Task Name         3103 non-null   object 
 16  Assignee          321 non-null    object 


In [11]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [21]:
llm = OpenAI(temperature=0,
             #api_token=OPENAI_API_KEY
            )


In [46]:
base_df['Creative Brief'].value_counts()

Eels wrap around the cops arm and make him shoot the gun                                               24
WHAT ARE WE DOING HERE?                                                                                22
Good for MW shadow mockup                                                                              20
NEEDS TEMP                                                                                             20
Eels jump onto cop one sync with Jacob's hand                                                          16
                                                                                                       ..
Footage needs to look messed up and loz rez on purpose                                                  1
Remove wire and fix the timming on the Red nose light to match the scare moment.                        1
Darken right side to match previous cut                                                                 1
Look a crew member in the back of the glass an

In [42]:
column_dtype_list = [(column, str(dtype)) for column, dtype in base_df.dtypes.items()]
columns_str = ", ".join([f"{col}({dtype})" for col, dtype in column_dtype_list])
final_str = f"The columns are {columns_str}."
print(final_str)


The columns are Id(int64), Thumbnail(object), Shot-Code(object), Status(object), Description(object), Type(object), Misc(object), Creative Brief(object), Reel(int64), Vendor-Groups(object), Priority(float64), Open Notes Count(int64), Version-Link(object), Id.1(float64), Status.1(object), Task Name(object), Assignee(object), Start Date(float64), Due-Date(float64), Duration(float64), Turnover(object), Sequence(object), Task-Template(object), Project(object).


In [75]:
filtered_ids = base_df[(base_df['Assignee'] == 'Jose Marra') & (base_df['Shot-Code'] == 'LA_0070')]['Id'].tolist()
filtered_ids

[1189]

In [76]:
base_df.loc[base_df['Id']==1189]

,Id,Thumbnail,Shot-Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor-Groups,Priority,Open Notes Count,Version-Link,Id.1,Status.1,Task Name,Assignee,Start Date,Due-Date,Duration,Turnover,Sequence,Task-Template,Project
0,1189,NaN,LA_0070,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9055.0,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper


In [77]:
template_old="""
You have to rephrase the provided input into a prompt, that will be passed to a python agent. Actually, goal is to get the desired output/s from the given csv.
But the first input can be a general one, so you need to rephrase prompt in such way, that it is communicating the highlevel requirements on specific columns.
The columns are Id(int64), Thumbnail(object), Shot-Code(object), Status(object), Description(object), Type(object), Misc(object),\
Creative Brief(object), Reel(int64), Vendor-Groups(object), Priority(float64), Open Notes Count(int64), Version-Link(object),\
Id.1(float64), Status.1(object), Task Name(object), Assignee(object), Start Date(float64), Due-Date(float64), Duration(float64),\
Turnover(object), Sequence(object), Task-Template(object), Project(object).
Let me provide you some examples:
text:I would like to identify the biggest motor purchasers from my company that are not resellers.
response: Get the customer_name and company_id for orders that have highest sales_revenue for product_category_name containing motor, irrespective of case. And reseller status should be False. Give me top 5

Let me provide you some examples:
text:I would like to identify the workload on each member
response: Give me value_counts for column "Assigned To"

text: give me ids of all those tasks assigned to Jose Marra, and have shot code: LA_0070
response ; df[(df['Assignee'] == 'Jose Marra') & (df['Shot-Code'] == 'LA_0070')]['Id'].tolist()

text: {input}
"""

In [70]:
template=f"""
This dataframe is about a VFX-Shots of a film. Like we have clip ids, and then we have different attributes, like Assignees, Creative Brief(description of clip).
You have to rephrase the provided input into a prompt, that will be passed to a python agent. Actually, goal is to get the desired output/s from the given csv.
But the first input can be a general one, so you need to rephrase prompt in such way, that it is communicating the highlevel requirements on specific columns.
The columns are Id(int64), Thumbnail(object), Shot-Code(object), Status(object), Description(object), Type(object), Misc(object),\
Creative Brief(object), Reel(int64), Vendor-Groups(object), Priority(float64), Open Notes Count(int64), Version-Link(object),\
Id.1(float64), Status.1(object), Task Name(object), Assignee(object), Start Date(float64), Due-Date(float64), Duration(float64),\
Turnover(object), Sequence(object), Task-Template(object), Project(object).

Assignee : The persons the clips are assigned to
Creative Brief : Is the description/comments on what is in the clip

Let me provide you some examples:
text:I would like to identify the workload on each member
response: Give me value_counts for column "Assigned To"



text: {input}
"""
#llm = OpenAI(temperature=0,api_token=OPENAI_API_KEY)
prompt_template = PromptTemplate.from_template(template=template_old)
chain = LLMChain(llm=llm, prompt=prompt_template)
#to_show_df=pd.read_csv(large_files_pth/'GolfProject/19June-focused-merged.csv')
#basic_agent = create_csv_agent(OpenAI(temperature=0),str(large_files_pth/'GolfProject/19June-focused-merged.csv'),verbose=True,max_iterations=7)
# 27 July, change, having shipping city



In [56]:
base_df.head(3)

,Id,Thumbnail,Shot-Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor-Groups,Priority,Open Notes Count,Version-Link,Id.1,Status.1,Task Name,Assignee,Start Date,Due-Date,Duration,Turnover,Sequence,Task-Template,Project
0,1189,NaN,LA_0070,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9055.0,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
1,1190,NaN,LA_0080,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9056.0,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper
2,1191,NaN,LA_0100,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9057.0,wtg,Graphics,Jose Marra,NaN,NaN,NaN,No,LA - Liz Apartment,NaN,Mister Whisper


In [58]:
cleaned_df = base_df.dropna(axis=1, how='all')
cleaned_df

,Id,Thumbnail,Shot-Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor-Groups,Priority,Open Notes Count,Version-Link,Id.1,Status.1,Task Name,Assignee,Turnover,Sequence,Task-Template,Project
0,1189,NaN,LA_0070,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9055.0,wtg,Graphics,Jose Marra,No,LA - Liz Apartment,NaN,Mister Whisper
1,1190,NaN,LA_0080,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9056.0,wtg,Graphics,Jose Marra,No,LA - Liz Apartment,NaN,Mister Whisper
2,1191,NaN,LA_0100,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9057.0,wtg,Graphics,Jose Marra,No,LA - Liz Apartment,NaN,Mister Whisper
3,1193,NaN,LA_0120,wtg,Website graphics / Screen capture,Graphics,NaN,NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE,1,"Graphic Artist, Jose VFX",1.0,0,NaN,9058.0,wtg,Graphics,Jose Marra,No,LA - Liz Apartment,NaN,Mister Whisper
4,1295,https://misterwhisper.shotgrid.autodesk.com/th...,CR_0350,wtg,2D Shadow comp of MW in BG,2D Comp,NaN,MW shadow continues to grow over the last pane...,2,"Eye-Spy VFX, In-house",1.0,1,CR_0350_Ref_v01,8004.0,wtg,Roto,NaN,No,CR - Chaz Rescue,2D Comp,Mister Whisper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",NaN,0,LH_0025_Ref_v01,6307.0,wtg,Roto,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3635,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",NaN,0,LH_0025_Ref_v01,6306.0,wtg,Comp,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3636,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",NaN,0,LH_0025_Ref_v01,6305.0,wtg,Depth Map,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper
3637,2826,https://misterwhisper.shotgrid.autodesk.com/th...,LH_0025,quest,Bubble Jacob,3D Comp L1,NaN,Does Whisper talk here? Is it muffled? Does t...,3,"Eye-Spy VFX, In-house",NaN,0,LH_0025_Ref_v01,6304.0,wtg,Animation,NaN,NaN,LH - Liz Hides,3D Comp - L1,Mister Whisper


In [61]:
len(cleaned_df.columns), len(base_df.columns)

(21, 24)

In [63]:
sample_df = cleaned_df.dropna().sample(n=2)
print(sample_df)


       Id                                          Thumbnail Shot-Code Status  \
492  1600  https://misterwhisper.shotgrid.autodesk.com/th...   AJ_0170  quest   
392  1575  https://misterwhisper.shotgrid.autodesk.com/th...   AJ_0420    wtg   

                                 Description        Type  \
492                           Shadow on wall  3D Comp L1   
392  Add shadows to Adam's body and shotgun.  3D Comp L2   

                               Misc  \
492       What is the shadow doing?   
392  Is this shot still in the cut?   

                                        Creative Brief  Reel  \
492                                              ?????     3   
392  Shadows added to Adam's body. They are not adv...     3   

             Vendor-Groups  Priority  Open Notes Count     Version-Link  \
492               In-house       1.0                 0  AJ_0170_ref_v01   
392  Eye-Spy VFX, In-house       1.0                 1  AJ_0420_ref_v01   

       Id.1 Status.1     Task Name    

In [67]:
sample_df

,Id,Thumbnail,Shot-Code,Status,Description,Type,Misc,Creative Brief,Reel,Vendor-Groups,Priority,Open Notes Count,Version-Link,Id.1,Status.1,Task Name,Assignee,Turnover,Sequence,Task-Template,Project
492,1600,https://misterwhisper.shotgrid.autodesk.com/th...,AJ_0170,quest,Shadow on wall,3D Comp L1,What is the shadow doing?,?????,3,In-house,1.0,0,AJ_0170_ref_v01,6034.0,wtg,Animation,In-house,Maybe,AJ - Adam Jacob,3D Comp - L1,Mister Whisper
392,1575,https://misterwhisper.shotgrid.autodesk.com/th...,AJ_0420,wtg,Add shadows to Adam's body and shotgun.,3D Comp L2,Is this shot still in the cut?,Shadows added to Adam's body. They are not adv...,3,"Eye-Spy VFX, In-house",1.0,1,AJ_0420_ref_v01,6448.0,wtg,Object Track,Ratheesh Vasu,Yes,AJ - Adam Jacob,3D Comp - L2,Mister Whisper


In [64]:
basic_agent = create_csv_agent(OpenAI(temperature=0.2),
                               str('/home/ubuntu/workspace/Temp/07-14Oct/processed_test_df.csv'),
                               df_head = sample_df,
                               verbose=True,max_iterations=4)


In [65]:
basic_agent.__dict__

{'memory': None,
 'callbacks': None,
 'callback_manager': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'agent': ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   Id | Thumbnail                                                                     | Shot-Code   | Status   | Description                       | Type     |   Misc | Creative Brief                                                                                                                                                                       |   Reel | Vendor-Groups            |   Priority |   Open Notes Count | Version-Link    |   Id.1 | Status.1   | Task Name   | Assignee   |   Start Date |   Due-Date |   Duration | Turnover   | Sequence           | Task-Template   | Project        |\n|---:|-----:|:------------------------------------------------------------------------------|:------------|:---------|:----------------

In [71]:
#Fns
def full_flow(query,rephrase_chain = chain, action_agent=basic_agent):
    rephrased=rephrase_chain.predict(input=query)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(rephrased)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
    final_res = action_agent.run(rephrased)

    return rephrased,final_res

In [25]:
reph, res = full_flow('what is the workload on every assigne')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me the count of orders for each value in the column "Assigned To"
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to count the number of orders for each value in the Assigned To column
Action: python_repl_ast
Action Input: df['Assignee'].value_counts()
Observation: Ratheesh Vasu      160
Mike Linman         75
Jose Marra          43
In-house            23
Eye-Spy VFX         13
Robert Glickert      4
Jose VFX             1
Merrin Marra         1
George Moise         1
Name: Assignee, dtype: int64
Thought: I now know the final answer
Final Answer: Ratheesh Vasu has 160 orders, Mike Linman has 75 orders, Jose Marra has 43 orders, In-house has 23 orders, Eye-Spy VFX has 13 orders, Robert Glickert has 4 orders, Jose VFX has 1 order, Merrin Marra has 1 order, and George Moise has 1 order.

> Finished chain.


In [26]:
res

'Ratheesh Vasu has 160 orders, Mike Linman has 75 orders, Jose Marra has 43 orders, In-house has 23 orders, Eye-Spy VFX has 13 orders, Robert Glickert has 4 orders, Jose VFX has 1 order, Merrin Marra has 1 order, and George Moise has 1 order.'

In [27]:
reph, res = full_flow('describe me, what is this dataframe about')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me a summary of the columns in the csv, including data types and counts of unique values.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to get a summary of the dataframe
Action: python_repl_ast
Action Input: df.info()
Observation: <class 'pandas.core.frame.DataFrame'>
RangeIndex: 3639 entries, 0 to 3638
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3639 non-null   int64  
 1   Thumbnail         3588 non-null   object 
 2   Shot-Code         3639 non-null   object 
 3   Status            3623 non-null   object 
 4   Description       3516 non-null   object 
 5   Type              3639 non-null   object 
 6   Misc              171 non-null    object 
 7   Creative Brief    691 non-null    object 
 8   Reel              3639 non-null   int64  
 9   Vendor-Groups     3639 non-null   objec

In [28]:
reph, res = full_flow('what assignee has top incoming due dates, give me top 10')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me the top 10 assignees with the highest number of incoming due dates in descending order.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to find the assignees with the highest number of due dates.
Action: python_repl_ast
Action Input: df.groupby('Assignee')['Due-Date'].count().sort_values(ascending=False).head(10)
Observation: Assignee
Eye-Spy VFX        0
George Moise       0
In-house           0
Jose Marra         0
Jose VFX           0
Merrin Marra       0
Mike Linman        0
Ratheesh Vasu      0
Robert Glickert    0
Name: Due-Date, dtype: int64
Thought: I now know the final answer
Final Answer: The top 10 assignees with the highest number of incoming due dates in descending order are Eye-Spy VFX, George Moise, In-house, Jose Marra, Jose VFX, Merrin Marra, Mike Linman, Ratheesh Vasu, and Robert Glickert.

> Finished chain.


In [29]:
res

'The top 10 assignees with the highest number of incoming due dates in descending order are Eye-Spy VFX, George Moise, In-house, Jose Marra, Jose VFX, Merrin Marra, Mike Linman, Ratheesh Vasu, and Robert Glickert.'

In [30]:
reph, res = full_flow('what assignee has top incoming due dates, give me top 10, also give me the due-dates')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me the top 10 due dates for each assignee in descending order.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to sort the dataframe by assignee and due date.
Action: python_repl_ast
Action Input: df.sort_values(by=['Assignee', 'Due-Date'], ascending=False).head(10)
Observation:         Id                                          Thumbnail Shot-Code  \
1845  1375  https://misterwhisper.shotgrid.autodesk.com/th...   CJ_0010   
2076  1350  https://misterwhisper.shotgrid.autodesk.com/th...   CJ_0110   
2424  1623  https://misterwhisper.shotgrid.autodesk.com/th...   CA_0250   
3500  2478  https://misterwhisper.shotgrid.autodesk.com/th...   CJ_0125   
30    1336  https://misterwhisper.shotgrid.autodesk.com/th...   CJ_0250   
32    1336  https://misterwhisper.shotgrid.autodesk.com/th...   CJ_0250   
93    1489  https://misterwhisper.shotgrid.autodesk.com/th...   LH_0170   
111   1494  https://m

In [31]:
res

'The top 10 due dates for each assignee in descending order are: Robert Glickert: CJ_0010_Ref_v01, CJ_0110_Ref_v01, CA_0250_ref_v01, CJ_0125_Ref_v01, Ratheesh Vasu: CJ_0250_Ref_v01, LH_0170_ref_v01, LH_0120_ref_v01, LH_0090_ref_v01, AK_0410_ref_v01.'

In [ ]:
LA_0070

In [35]:
reph, res = full_flow('give me ids of all those tasks assigned to Jose Marra, and have shot code: LA_0070')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me the order_ids for customer_name "Jose Marra" and product_category_name "LA_0070".
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to get the rows with the customer_name and product_category_name.
Action: python_repl_ast
Action Input: df[(df['Creative Brief'] == 'Jose Marra') & (df['Shot-Code'] == 'LA_0070')]['Id']
Observation: Series([], Name: Id, dtype: int64)
Thought: This is not the answer I was expecting.
Action: python_repl_ast
Action Input: df[(df['Creative Brief'] == 'Jose Marra') & (df['Shot-Code'] == 'LA_0070')]['Id'].values
Observation: []
Thought: This is still not the answer I was expecting.
Action: python_repl_ast
Action Input: df[(df['Creative Brief'] == 'Jose Marra') & (df['Shot-Code'] == 'LA_0070')]['Id'].unique()
Observation: []
Thought: This is still not the answer I was expecting.
Action: python_repl_ast
Action Input: df[(df['Creative Brief'] 

In [36]:
res

'There are no order_ids for customer_name "Jose Marra" and product_category_name "LA_0070".'

In [50]:
reph, res = full_flow('give me ids of all those tasks assigned to Jose Marra, and have shot code: LA_0070')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me the unique values in the column "Creative Brief"
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to find the unique values in the column "Creative Brief"
Action: python_repl_ast
Action Input: df['Creative Brief'].unique()
Observation: ['NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE'
 'MW shadow continues to grow over the last panel and finishes filling up the wall one or two beats before Jacob looks up. We need to leave the last window on the right open and lit.'
 'The sadow of MW continues to grow over the wall, filling up the second panel (Middle of frame)'
 'A shadow grows over the left side punching bag. It is MW slowly growing over the very back wall.'
 'Jacob in shadows approaches camera. We see a glint in his eyes. Behind him we can see/sense the presence of MW. We see a thin shadow torso behind Jacob. The wall can light up and down but MW needs to ne in the frame the whole time.

In [51]:
res

'The unique values in the column "Creative Brief" are: NEED TO FIGURE OUT WHAT THE SCREEN DOES HERE, MW shadow continues to grow over the last panel and finishes filling up the wall one or two beats before Jacob looks up. We need to leave the last window on the right open and lit., The sadow of MW continues to grow over the wall, filling up the second panel (Middle of frame), A shadow grows over the left side punching bag. It is MW slowly growing over the very back wall., Jacob in shadows approaches camera. We see a glint in his eyes. Behind him we can see/sense the presence of MW. We see a thin shadow torso behind Jacob. The wall can light up and down but MW needs to ne in the frame the whole time., MW needs to reach the top of the wall by this point., The shadow of MW is growing behind Jacob towards the ceiling., nan, We need to be able to set more than one level of lighting manipulation and also separate the different lights like the lamp and the wall., Lights start to breathe in a 

In [72]:
reph, res = full_flow('what is the workload on every assigne')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me value_counts for column "Assignee"
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to find the counts of each value in the Assignee column
Action: python_repl_ast
Action Input: df['Assignee'].value_counts()
Observation: Ratheesh Vasu      160
Mike Linman         75
Jose Marra          43
In-house            23
Eye-Spy VFX         13
Robert Glickert      4
Jose VFX             1
Merrin Marra         1
George Moise         1
Name: Assignee, dtype: int64
Thought: I now know the final answer
Final Answer: Ratheesh Vasu has the most Assignee values with 160.

> Finished chain.


In [78]:
reph, res = full_flow('give me ids of all those tasks assigned to Jose Marra, and have shot code: LA_0070')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Get the Ids of all the tasks with Assignee as Jose Marra and Shot-Code as LA_0070.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to get the tasks with Assignee as Jose Marra and Shot-Code as LA_0070.
Action: python_repl_ast
Action Input: df[(df['Assignee'] == 'Jose Marra') & (df['Shot-Code'] == 'LA_0070')]['Id']
Observation: 0    1189
Name: Id, dtype: int64
Thought: I now know the final answer
Final Answer: 1189

> Finished chain.


In [79]:
res

'1189'

In [80]:
reph, res = full_flow('give me workloads of every person')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
response: Give me value_counts for column "Assignee"
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


> Entering new AgentExecutor chain...
Thought: I need to get the value_counts for the Assignee column
Action: python_repl_ast
Action Input: df['Assignee'].value_counts()
Observation: Ratheesh Vasu      160
Mike Linman         75
Jose Marra          43
In-house            23
Eye-Spy VFX         13
Robert Glickert      4
Jose VFX             1
Merrin Marra         1
George Moise         1
Name: Assignee, dtype: int64
Thought: I now know the final answer
Final Answer: Ratheesh Vasu has the most assignments with 160.

> Finished chain.


In [81]:
res

'Ratheesh Vasu has the most assignments with 160.'